In [9]:
# Helper function for iterative text generation using a fill-mask pipeline
def generate_text_with_mask(pipeline, prompt, mask_token, num_steps=5):
    generated_text = prompt
    for _ in range(num_steps):
        text_with_mask = generated_text + " " + mask_token # Add space and mask token
        predictions = pipeline(text_with_mask)
        if not predictions:
            break
        # Take the top prediction token and append it
        next_token = predictions[0]['token_str'].strip()

        # Handle punctuation to avoid extra spaces
        if next_token in [" .", " ,", " !", " ?", " ;"]: # RoBERTa might predict with leading space
            generated_text += next_token.replace(" ", "")
        elif next_token in [".", ",", "!", "?", ";"]:
            generated_text += next_token
        elif generated_text.endswith(tuple([" .", " ,", " !", " ?", " ;"])):
            # If previous token was punctuation, don't add space
            generated_text += next_token
        else:
            generated_text += " " + next_token

        # Prevent infinite loops if the model keeps predicting the same thing or padding tokens
        if next_token == mask_token or next_token == "[PAD]" or next_token == "</s>":
            break
    return generated_text.strip()

## BERT model

In [1]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModelForQuestionAnswering, pipeline

# 2. Define the model name as 'bert-base-uncased'
model_name = 'bert-base-uncased'

# 3. Load the tokenizer for the defined model
tokenizer = AutoTokenizer.from_pretrained(model_name)
print(f"Tokenizer '{model_name}' loaded successfully.")

# 4. Load the model for masked language modeling
model_masked_lm = AutoModelForMaskedLM.from_pretrained(model_name)
print(f"Masked Language Model '{model_name}' loaded successfully.")

# 5. Load the model for question answering
model_qa = AutoModelForQuestionAnswering.from_pretrained(model_name)
print(f"Question Answering Model '{model_name}' loaded successfully.")

# 6. Create a pipeline for 'fill-mask' tasks
fill_mask_pipeline = pipeline("fill-mask", model=model_masked_lm, tokenizer=tokenizer)
print("Fill-mask pipeline created successfully.")

# 7. Create a pipeline for 'question-answering' tasks
qa_pipeline = pipeline("question-answering", model=model_qa, tokenizer=tokenizer)
print("Question-answering pipeline created successfully.")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


Masked Language Model 'bert-base-uncased' loaded successfully.
Question Answering Model 'bert-base-uncased' loaded successfully.


Device set to use cuda:0


Fill-mask pipeline created successfully.
Question-answering pipeline created successfully.


In [10]:
print("\n--- Performing Masked Prediction and Text Generation with BERT ---")

sentence1 = "The future of Artificial Intelligence is"
print(f"\nGenerating text for: '{sentence1}'")
generated_text1 = generate_text_with_mask(fill_mask_pipeline, sentence1, '[MASK]')
print(f"  Generated: '{generated_text1}'")

sentence2 = "The goal of Generative AI is to [MASK] new content."
print(f"\nPredicting for: '{sentence2}'")
predictions2 = fill_mask_pipeline(sentence2)
for p in predictions2:
    print(f"  Token: {p['token_str']}, Score: {p['score']:.4f}")


--- Performing Masked Prediction and Text Generation with BERT ---

Generating text for: 'The future of Artificial Intelligence is'
  Generated: 'The future of Artificial Intelligence is.....'

Predicting for: 'The goal of Generative AI is to [MASK] new content.'
  Token: create, Score: 0.5397
  Token: generate, Score: 0.1558
  Token: produce, Score: 0.0541
  Token: develop, Score: 0.0445
  Token: add, Score: 0.0176


- BERT predicted the token like '.' indicating it recognises the need forending the sentence.
- BERT predicted words like 'create', 'generate', and 'produce'.

In [3]:
print("\n--- Performing Question Answering ---")

context = "Generative AI poses significant risks such as hallucinations, bias, and deepfakes."
question = "What are the risks?"

print(f"\nContext: '{context}'")
print(f"Question: '{question}'")

qa_result = qa_pipeline(question=question, context=context)
print(f"Answer: {qa_result['answer']}")
print(f"Score: {qa_result['score']:.4f}")


--- Performing Question Answering ---

Context: 'Generative AI poses significant risks such as hallucinations, bias, and deepfakes.'
Question: 'What are the risks?'
Answer: AI poses significant risks
Score: 0.0048


BERT gets the context from the string that AI poses risks but does not mention what the risks are.

## RoBERTa model

In [4]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModelForQuestionAnswering, pipeline

# Define the model name as 'roberta-base'
model_name_roberta = 'roberta-base'

# Load the tokenizer for the defined model
tokenizer_roberta = AutoTokenizer.from_pretrained(model_name_roberta)
print(f"Tokenizer '{model_name_roberta}' loaded successfully.")

# Load the model for masked language modeling
model_masked_lm_roberta = AutoModelForMaskedLM.from_pretrained(model_name_roberta)
print(f"Masked Language Model '{model_name_roberta}' loaded successfully.")

# Load the model for question answering
model_qa_roberta = AutoModelForQuestionAnswering.from_pretrained(model_name_roberta)
print(f"Question Answering Model '{model_name_roberta}' loaded successfully.")

# Create a pipeline for 'fill-mask' tasks
fill_mask_pipeline_roberta = pipeline("fill-mask", model=model_masked_lm_roberta, tokenizer=tokenizer_roberta)
print("RoBERTa fill-mask pipeline created successfully.")

# Create a pipeline for 'question-answering' tasks
qa_pipeline_roberta = pipeline("question-answering", model=model_qa_roberta, tokenizer=tokenizer_roberta)
print("RoBERTa question-answering pipeline created successfully.")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


Masked Language Model 'roberta-base' loaded successfully.
Question Answering Model 'roberta-base' loaded successfully.


Device set to use cuda:0


RoBERTa fill-mask pipeline created successfully.
RoBERTa question-answering pipeline created successfully.


In [11]:
print("\n--- Performing Masked Prediction and Text Generation with RoBERTa ---")

sentence1_roberta = "The future of Artificial Intelligence is"
print(f"\nGenerating text for: '{sentence1_roberta}'")
generated_text1_roberta = generate_text_with_mask(fill_mask_pipeline_roberta, sentence1_roberta, '<mask>')
print(f"  Generated: '{generated_text1_roberta}'")

sentence2_roberta = "The goal of Generative AI is to <mask> new content."
print(f"\nPredicting for: '{sentence2_roberta}'")
predictions2_roberta = fill_mask_pipeline_roberta(sentence2_roberta)
for p in predictions2_roberta:
    print(f"  Token: {p['token_str']}, Score: {p['score']:.4f}")


--- Performing Masked Prediction and Text Generation with RoBERTa ---

Generating text for: 'The future of Artificial Intelligence is'
  Generated: 'The future of Artificial Intelligence is uncertain. </s>'

Predicting for: 'The goal of Generative AI is to <mask> new content.'
  Token:  generate, Score: 0.3711
  Token:  create, Score: 0.3677
  Token:  discover, Score: 0.0835
  Token:  find, Score: 0.0213
  Token:  provide, Score: 0.0165


- RoBERTa predicted the next token as 'uncertain', better than BERT predicting a string of '.'

In [6]:
print("\n--- Performing Question Answering with RoBERTa ---")

context_roberta = "Generative AI poses significant risks such as hallucinations, bias, and deepfakes."
question_roberta = "What are the risks?"

print(f"\nContext: '{context_roberta}'")
print(f"Question: '{question_roberta}'")

qa_result_roberta = qa_pipeline_roberta(question=question_roberta, context=context_roberta)
print(f"Answer: {qa_result_roberta['answer']}")
print(f"Score: {qa_result_roberta['score']:.4f}")


--- Performing Question Answering with RoBERTa ---

Context: 'Generative AI poses significant risks such as hallucinations, bias, and deepfakes.'
Question: 'What are the risks?'
Answer: as hallucinations, bias, and deepfakes.
Score: 0.0052


RoBERTa extracted the right risks from the given context string

## BART

In [12]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModelForQuestionAnswering, pipeline

# Define the model name as 'facebook/bart-base'
model_name_bart = 'facebook/bart-base'

# Load the tokenizer for the defined model
tokenizer_bart = AutoTokenizer.from_pretrained(model_name_bart)
print(f"Tokenizer '{model_name_bart}' loaded successfully.")

# Load the model for masked language modeling
model_masked_lm_bart = AutoModelForMaskedLM.from_pretrained(model_name_bart)
print(f"Masked Language Model '{model_name_bart}' loaded successfully.")

# Load the model for question answering
model_qa_bart = AutoModelForQuestionAnswering.from_pretrained(model_name_bart)
print(f"Question Answering Model '{model_name_bart}' loaded successfully.")

# Create a pipeline for 'fill-mask' tasks
fill_mask_pipeline_bart = pipeline("fill-mask", model=model_masked_lm_bart, tokenizer=tokenizer_bart)
print("BART fill-mask pipeline created successfully.")

# Create a pipeline for 'question-answering' tasks
qa_pipeline_bart = pipeline("question-answering", model=model_qa_bart, tokenizer=tokenizer_bart)
print("BART question-answering pipeline created successfully.")

config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Tokenizer 'facebook/bart-base' loaded successfully.


model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

Some weights of BartForQuestionAnswering were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


Masked Language Model 'facebook/bart-base' loaded successfully.
Question Answering Model 'facebook/bart-base' loaded successfully.


Device set to use cuda:0


BART fill-mask pipeline created successfully.
BART question-answering pipeline created successfully.


In [15]:
print("\n--- Performing Masked Prediction and Text Generation with BART ---")

sentence1_bart = "The future of Artificial Intelligence is"
print(f"\nGenerating text for: '{sentence1_bart}'")
generated_text1_bart = generate_text_with_mask(fill_mask_pipeline_bart, sentence1_bart, '<mask>', 10)
print(f"  Generated: '{generated_text1_bart}'")

sentence2_bart = "The goal of Generative AI is to <mask> new content."
print(f"\nPredicting for: '{sentence2_bart}'")
predictions2_bart = fill_mask_pipeline_bart(sentence2_bart)
for p in predictions2_bart:
    print(f"  Token: {p['token_str']}, Score: {p['score']:.4f}")

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset



--- Performing Masked Prediction and Text Generation with BART ---

Generating text for: 'The future of Artificial Intelligence is'
  Generated: 'The future of Artificial Intelligence is in the hands of the people. advertisement : advertisement'

Predicting for: 'The goal of Generative AI is to <mask> new content.'
  Token:  create, Score: 0.0746
  Token:  help, Score: 0.0657
  Token:  provide, Score: 0.0609
  Token:  enable, Score: 0.0359
  Token:  improve, Score: 0.0332


In [14]:
print("\n--- Performing Question Answering with BART ---")

context_bart = "Generative AI poses significant risks such as hallucinations, bias, and deepfakes."
question_bart = "What are the risks?"

print(f"\nContext: '{context_bart}'")
print(f"Question: '{question_bart}'")

qa_result_bart = qa_pipeline_bart(question=question_bart, context=context_bart)
print(f"Answer: {qa_result_bart['answer']}")
print(f"Score: {qa_result_bart['score']:.4f}")


--- Performing Question Answering with BART ---

Context: 'Generative AI poses significant risks such as hallucinations, bias, and deepfakes.'
Question: 'What are the risks?'
Answer: poses significant risks such
Score: 0.0293


| Task | Model | Classification (Success/Failure) | Observation (What actually happened?) | Why did this happen? (Architectural Reason) |
| :--- | :--- | :--- | :--- | :--- |
| **Generation** | BERT | *Failure* | Generated '.....' (primarily punctuation). | BERT is an Encoder-only model, not designed for open-ended text generation. |
| | RoBERTa | *Success* | Generated 'uncertain. </s>' | RoBERTa is also Encoder-only but its training leads to more plausible iterative masked predictions. |
| | BART | *Partial Success* | Generated 'in the hands of the people.<br> advertisement : advertisement'.<br>More coherent than BERT, but still incomplete. | BART is an Encoder-Decoder model, designed for text generation, but `num_steps` limit and general training affect coherence. |
| **Fill-Mask** | BERT | *Success* | Predicted 'create', 'generate'. | BERT is explicitly trained on Masked Language Modeling (MLM). |
| | RoBERTa | *Success* | Predicted 'generate', 'create'. | RoBERTa is also explicitly trained on Masked Language Modeling (MLM). |
| | BART | *Partial Success* | Predicted 'create', 'help', 'provide'.<br>Lower scores and less focused top predictions. | BART uses masked inputs during pre-training, but its Encoder-Decoder architecture can influence prediction distribution. |
| **QA** | BERT | *Partial Success* | Answered 'AI poses significant risks' (Score: 0.0048). Did not list specific risks. | BERT-based QA extracts a span. It found a relevant, but incomplete, general span. |
| | RoBERTa | *Success* | Answered 'as hallucinations, bias, and deepfakes.'<br> (Score: 0.0052). Clearly listed specific risks. | RoBERTa-based QA extracts spans. It successfully identified the precise span containing the risks. |
| | BART | *Partial Success* | Answered 'poses significant risks such' (Score: 0.0293).<br>More specific than BERT, but incomplete compared to RoBERTa. | BART performs QA by span extraction/generation. It extracted a relevant, but incomplete, leading phrase. |